In [1]:
import pandas as pd
import numpy as np
import math
from pandas.tseries.offsets import DateOffset
from datetime import timedelta

In [2]:
df=pd.read_csv("Grocery_Inventory_and_Sales_Dataset.csv")
df.head()

,Product_ID,Product_Name,Catagory,Supplier_ID,Supplier_Name,Stock_Quantity,Reorder_Level,Reorder_Quantity,Unit_Price,Date_Received,Last_Order_Date,Expiration_Date,Warehouse_Location,Sales_Volume,Inventory_Turnover_Rate,Status
0,29-205-1132,Sushi Rice,Grains & Pulses,38-037-1699,Jaxnation,22,72,70,$4.50,8/16/2024,6/29/2024,9/19/2024,48 Del Sol Trail,32,19,Discontinued
1,40-681-9981,Arabica Coffee,Beverages,54-470-2479,Feedmix,45,77,2,$20.00,11/1/2024,5/29/2024,5/8/2024,36 3rd Place,85,1,Discontinued
2,06-955-3428,Black Rice,Grains & Pulses,54-031-2945,Vinder,30,38,83,$6.00,8/3/2024,6/10/2024,9/22/2024,3296 Walton Court,31,34,Backordered
3,71-594-6552,Long Grain Rice,Grains & Pulses,63-492-7603,Brightbean,12,59,62,$1.50,12/8/2024,2/19/2025,4/17/2024,3 Westerfield Crossing,95,99,Active
4,57-437-1828,Plum,Fruits & Vegetables,54-226-4308,Topicstorm,37,30,74,$4.00,7/3/2024,10/11/2024,10/5/2024,15068 Scoville Court,62,25,Backordered


In [3]:
df.isnull().sum()

Product_ID                 0
Product_Name               0
Catagory                   1
Supplier_ID                0
Supplier_Name              0
Stock_Quantity             0
Reorder_Level              0
Reorder_Quantity           0
Unit_Price                 0
Date_Received              0
Last_Order_Date            0
Expiration_Date            0
Warehouse_Location         0
Sales_Volume               0
Inventory_Turnover_Rate    0
Status                     0
dtype: int64

In [4]:
df=df.dropna()

In [5]:
df.isnull().sum()

Product_ID                 0
Product_Name               0
Catagory                   0
Supplier_ID                0
Supplier_Name              0
Stock_Quantity             0
Reorder_Level              0
Reorder_Quantity           0
Unit_Price                 0
Date_Received              0
Last_Order_Date            0
Expiration_Date            0
Warehouse_Location         0
Sales_Volume               0
Inventory_Turnover_Rate    0
Status                     0
dtype: int64

In [6]:
#del unwanted col
df = df.drop(['Last_Order_Date', 'Warehouse_Location','Sales_Volume','Inventory_Turnover_Rate','Status' ], axis=1)
len(df)

989

In [7]:
df.head()

,Product_ID,Product_Name,Catagory,Supplier_ID,Supplier_Name,Stock_Quantity,Reorder_Level,Reorder_Quantity,Unit_Price,Date_Received,Expiration_Date
0,29-205-1132,Sushi Rice,Grains & Pulses,38-037-1699,Jaxnation,22,72,70,$4.50,8/16/2024,9/19/2024
1,40-681-9981,Arabica Coffee,Beverages,54-470-2479,Feedmix,45,77,2,$20.00,11/1/2024,5/8/2024
2,06-955-3428,Black Rice,Grains & Pulses,54-031-2945,Vinder,30,38,83,$6.00,8/3/2024,9/22/2024
3,71-594-6552,Long Grain Rice,Grains & Pulses,63-492-7603,Brightbean,12,59,62,$1.50,12/8/2024,4/17/2024
4,57-437-1828,Plum,Fruits & Vegetables,54-226-4308,Topicstorm,37,30,74,$4.00,7/3/2024,10/5/2024


In [8]:
df['Expiration_Date'] = pd.to_datetime(df['Expiration_Date'], format='%m/%d/%Y',errors='coerce')
df['Date_Received'] = pd.to_datetime(df['Date_Received'], format='%m/%d/%Y',errors='coerce')

In [9]:
#exp
#curr_date=df['Date_Received'].max()
'''
curr_date='2025-01-01'
expired_items = df[df['Expiration_Date'] < curr_date]
not_exp=len(df)-expired_items.shape[0]
print(not_exp)
'''

"\ncurr_date='2025-01-01'\nexpired_items = df[df['Expiration_Date'] < curr_date]\nnot_exp=len(df)-expired_items.shape[0]\nprint(not_exp)\n"

In [10]:
#setting curr date as 2025-02-09
''' tables=expiered, discount, donation
create status in i_table

'''

' tables=expiered, discount, donation\ncreate status in i_table\n\n'

In [11]:
condition = df['Date_Received'] < pd.Timestamp('2024-11-01')

# Apply 7-month shift only to matching rows
df.loc[condition, 'Date_Received'] = df.loc[condition, 'Date_Received'] + pd.DateOffset(months=7)
df.loc[condition, 'Expiration_Date'] = df.loc[condition, 'Expiration_Date'] + pd.DateOffset(months=7)

# View result
df[['Date_Received', 'Expiration_Date']].head()

,Date_Received,Expiration_Date
0,2025-03-16,2025-04-19
1,2024-11-01,2024-05-08
2,2025-03-03,2025-04-22
3,2024-12-08,2024-04-17
4,2025-02-03,2025-05-05


In [12]:
print(df['Date_Received'].max())
print(df['Date_Received'].min())

2025-05-31 00:00:00
2024-09-25 00:00:00


In [13]:
# Calculate shelf life in days
df['Shelf_Life_(Days)'] = (df['Expiration_Date'] - df['Date_Received']).dt.days

# Swap values of exp and rec dates only where Shelf_Life_(Days) < 0
mask = df['Shelf_Life_(Days)'] < 0
# Use .loc to safely swap the values conditionally
df.loc[mask, ['Date_Received', 'Expiration_Date']] = df.loc[mask, ['Expiration_Date', 'Date_Received']].values
df['Shelf_Life_(Days)'] = (df['Expiration_Date'] - df['Date_Received']).dt.days
df['Shelf_Life_(Years)'] = (df['Shelf_Life_(Days)']/365).round(2)

max_years = math.ceil(df['Shelf_Life_(Years)'].max())
bins = list(range(0, max_years + 1))
labels = [f'{i}-{i+1} years' for i in range(0, max_years)]
df['Shelf_Life_Bin'] = pd.cut(df['Shelf_Life_(Years)'], bins=bins, labels=labels, right=False)
shelf_life_distribution = df['Shelf_Life_Bin'].value_counts().sort_index()

df.reset_index(drop=True,inplace=True)

# View a few rows
df.head()

,Product_ID,Product_Name,Catagory,Supplier_ID,Supplier_Name,Stock_Quantity,Reorder_Level,Reorder_Quantity,Unit_Price,Date_Received,Expiration_Date,Shelf_Life_(Days),Shelf_Life_(Years),Shelf_Life_Bin
0,29-205-1132,Sushi Rice,Grains & Pulses,38-037-1699,Jaxnation,22,72,70,$4.50,2025-03-16,2025-04-19,34,0.09,0-1 years
1,40-681-9981,Arabica Coffee,Beverages,54-470-2479,Feedmix,45,77,2,$20.00,2024-05-08,2024-11-01,177,0.48,0-1 years
2,06-955-3428,Black Rice,Grains & Pulses,54-031-2945,Vinder,30,38,83,$6.00,2025-03-03,2025-04-22,50,0.14,0-1 years
3,71-594-6552,Long Grain Rice,Grains & Pulses,63-492-7603,Brightbean,12,59,62,$1.50,2024-04-17,2024-12-08,235,0.64,0-1 years
4,57-437-1828,Plum,Fruits & Vegetables,54-226-4308,Topicstorm,37,30,74,$4.00,2025-02-03,2025-05-05,91,0.25,0-1 years


In [14]:
#making col for donation and discount dates in df

In [15]:
# Apply conditional logic for discount days:
# 20% of shelf life if bin is '0-1 years', otherwise 10%
discount_fraction = np.where(df['Shelf_Life_Bin'] == '0-1 years', 0.2, 0.1)
df['Discount_Date'] = df['Expiration_Date'] - pd.to_timedelta(df['Shelf_Life_(Days)'] * discount_fraction, unit='D')

print(df.head())

    Product_ID     Product_Name             Catagory  Supplier_ID  \
0  29-205-1132       Sushi Rice      Grains & Pulses  38-037-1699   
1  40-681-9981   Arabica Coffee            Beverages  54-470-2479   
2  06-955-3428       Black Rice      Grains & Pulses  54-031-2945   
3  71-594-6552  Long Grain Rice      Grains & Pulses  63-492-7603   
4  57-437-1828             Plum  Fruits & Vegetables  54-226-4308   

  Supplier_Name  Stock_Quantity  Reorder_Level  Reorder_Quantity Unit_Price  \
0     Jaxnation              22             72                70     $4.50    
1       Feedmix              45             77                 2    $20.00    
2        Vinder              30             38                83     $6.00    
3    Brightbean              12             59                62     $1.50    
4    Topicstorm              37             30                74     $4.00    

  Date_Received Expiration_Date  Shelf_Life_(Days)  Shelf_Life_(Years)  \
0    2025-03-16      2025-04-19     

In [16]:
# Set donation fraction based on shelf life bin
donation_fraction = np.where(df['Shelf_Life_Bin'] == '0-1 years', 0.1, 0.05)
df['Donation_Date'] = df['Expiration_Date'] - pd.to_timedelta(df['Shelf_Life_(Days)'] * donation_fraction, unit='D')

df.head()

,Product_ID,Product_Name,Catagory,Supplier_ID,Supplier_Name,Stock_Quantity,Reorder_Level,Reorder_Quantity,Unit_Price,Date_Received,Expiration_Date,Shelf_Life_(Days),Shelf_Life_(Years),Shelf_Life_Bin,Discount_Date,Donation_Date
0,29-205-1132,Sushi Rice,Grains & Pulses,38-037-1699,Jaxnation,22,72,70,$4.50,2025-03-16,2025-04-19,34,0.09,0-1 years,2025-04-12 04:48:00,2025-04-15 14:24:00
1,40-681-9981,Arabica Coffee,Beverages,54-470-2479,Feedmix,45,77,2,$20.00,2024-05-08,2024-11-01,177,0.48,0-1 years,2024-09-26 14:24:00,2024-10-14 07:12:00
2,06-955-3428,Black Rice,Grains & Pulses,54-031-2945,Vinder,30,38,83,$6.00,2025-03-03,2025-04-22,50,0.14,0-1 years,2025-04-12 00:00:00,2025-04-17 00:00:00
3,71-594-6552,Long Grain Rice,Grains & Pulses,63-492-7603,Brightbean,12,59,62,$1.50,2024-04-17,2024-12-08,235,0.64,0-1 years,2024-10-22 00:00:00,2024-11-14 12:00:00
4,57-437-1828,Plum,Fruits & Vegetables,54-226-4308,Topicstorm,37,30,74,$4.00,2025-02-03,2025-05-05,91,0.25,0-1 years,2025-04-16 19:12:00,2025-04-25 21:36:00


In [17]:
curr_date=df['Date_Received'].max()

In [18]:
# Inventory Table
inventory_table = df[['Product_ID', 'Product_Name', 'Catagory', 'Supplier_ID', 'Supplier_Name', 'Stock_Quantity', 'Reorder_Level', 'Reorder_Quantity','Unit_Price','Date_Received','Expiration_Date']]

In [19]:
#Discount Table
condition = (df['Discount_Date'] <= curr_date) & (curr_date < df['Donation_Date'])
selected_columns = ['Product_ID', 'Product_Name', 'Catagory', 'Supplier_ID', 'Supplier_Name', 'Stock_Quantity', 'Reorder_Level', 'Reorder_Quantity','Unit_Price','Date_Received','Expiration_Date','Discount_Date']

discount_table = df.loc[condition, selected_columns]
discount_table.head()

,Product_ID,Product_Name,Catagory,Supplier_ID,Supplier_Name,Stock_Quantity,Reorder_Level,Reorder_Quantity,Unit_Price,Date_Received,Expiration_Date,Discount_Date
40,11-316-8405,Mango,Fruits & Vegetables,50-918-1373,Twitternation,24,55,51,$5.00,2024-11-26,2025-06-25,2025-05-13 19:12:00
54,36-918-2937,Green Coffee,Beverages,53-844-0478,Kwinu,62,89,18,$12.00,2024-10-12,2025-06-28,2025-05-07 04:48:00
105,64-478-4745,Lemon,Fruits & Vegetables,36-780-8625,Mydo,63,81,94,$2.50,2025-01-04,2025-06-19,2025-05-16 19:12:00
162,76-325-9093,Sesame Oil,Oils & Fats,67-922-3399,Feedbug,27,92,56,$6.50,2024-11-18,2025-07-18,2025-05-30 14:24:00
207,00-215-7434,Egg (Goose),Dairy,85-272-3969,Quatz,14,63,74,$2.50,2024-11-28,2025-06-28,2025-05-16 14:24:00


In [20]:
#Donation Table
condition = (df['Donation_Date'] <= curr_date) & (df['Expiration_Date'] > curr_date)
selected_columns = ['Product_ID', 'Product_Name', 'Catagory', 'Supplier_ID', 'Supplier_Name', 'Stock_Quantity', 'Reorder_Level', 'Reorder_Quantity','Unit_Price','Date_Received','Expiration_Date','Donation_Date']

donation_table = df.loc[condition, selected_columns]
donation_table.head()

,Product_ID,Product_Name,Catagory,Supplier_ID,Supplier_Name,Stock_Quantity,Reorder_Level,Reorder_Quantity,Unit_Price,Date_Received,Expiration_Date,Donation_Date
45,90-230-9767,Long Grain Rice,Grains & Pulses,70-515-9048,Voonyx,67,88,32,$1.50,2024-11-06,2025-06-17,2025-05-25 16:48:00
149,19-967-0632,Short Grain Rice,Grains & Pulses,90-445-1753,Yombu,53,86,54,$3.00,2025-03-14,2025-06-06,2025-05-28 14:24:00
169,10-002-6494,Mango,Fruits & Vegetables,47-509-7282,Meevee,89,92,62,$4.80,2024-10-02,2025-06-11,2025-05-16 19:12:00
190,98-064-4465,Mozzarella Cheese,Dairy,93-719-9817,Feedfire,96,38,17,$7.00,2024-11-19,2025-06-11,2025-05-21 14:24:00
205,73-118-1117,Olive Oil,Oils & Fats,55-557-8544,Layo,52,88,55,$6.00,2024-12-27,2025-06-13,2025-05-27 04:48:00


In [21]:
#Expired Table
condition =  (df['Expiration_Date'] <= curr_date)
selected_columns = ['Product_ID', 'Product_Name', 'Catagory', 'Supplier_ID', 'Supplier_Name', 'Stock_Quantity', 'Reorder_Level', 'Reorder_Quantity','Unit_Price','Date_Received','Expiration_Date']

expired_table = df.loc[condition, selected_columns]
expired_table.head()

,Product_ID,Product_Name,Catagory,Supplier_ID,Supplier_Name,Stock_Quantity,Reorder_Level,Reorder_Quantity,Unit_Price,Date_Received,Expiration_Date
0,29-205-1132,Sushi Rice,Grains & Pulses,38-037-1699,Jaxnation,22,72,70,$4.50,2025-03-16,2025-04-19
1,40-681-9981,Arabica Coffee,Beverages,54-470-2479,Feedmix,45,77,2,$20.00,2024-05-08,2024-11-01
2,06-955-3428,Black Rice,Grains & Pulses,54-031-2945,Vinder,30,38,83,$6.00,2025-03-03,2025-04-22
3,71-594-6552,Long Grain Rice,Grains & Pulses,63-492-7603,Brightbean,12,59,62,$1.50,2024-04-17,2024-12-08
4,57-437-1828,Plum,Fruits & Vegetables,54-226-4308,Topicstorm,37,30,74,$4.00,2025-02-03,2025-05-05


In [22]:
#Fresh Table
condition =  (df['Discount_Date'] > curr_date)
selected_columns = ['Product_ID', 'Product_Name', 'Catagory', 'Supplier_ID', 'Supplier_Name', 'Stock_Quantity', 'Reorder_Level', 'Reorder_Quantity','Unit_Price','Date_Received','Expiration_Date']

fresh_table = df.loc[condition, selected_columns]
fresh_table.head()

,Product_ID,Product_Name,Catagory,Supplier_ID,Supplier_Name,Stock_Quantity,Reorder_Level,Reorder_Quantity,Unit_Price,Date_Received,Expiration_Date
19,76-854-0095,White Sugar,Grains & Pulses,38-883-3382,Mydeo,65,61,82,$2.00,2024-12-31,2025-07-28
28,31-211-5803,Oatmeal Biscuit,Bakery,75-503-9347,Gigazoom,16,9,32,$5.00,2025-02-16,2025-09-07
29,16-499-5059,Pear,Fruits & Vegetables,09-090-6360,Centizu,77,20,5,$4.50,2025-03-18,2025-07-20
34,73-401-5721,Haddock,Seafood,38-622-5251,Midel,46,28,73,$9.00,2024-12-28,2025-07-26
36,54-109-8062,Sushi Rice,Grains & Pulses,04-726-3363,Linklinks,75,69,100,$4.50,2025-02-11,2025-09-16


In [23]:
len(fresh_table)+len(expired_table)+len(discount_table)+len(donation_table)

989

In [24]:
len(df)

989